# Imaging biomarker extraction walkthrough
This notebook will go over some examples of extacting biomarkers from structural T1 weighted MR images and amyloid PET data. These are some of the most widely used biomarkers that are used in AD research. They are also measures used in the EBM. The walkthrough will use some freely available, widely used open-source software packages as an example of how to perform this analysis. However, there are many other methods out there to choose from and determine which one would be right for your data and application.

## Preparation
First up, we need to import some packages into our notebook so that we can use them. For the purposes of this notebook, we will use [nipype](https://nipype.readthedocs.io/en/latest/) a handy workflow engine in Python that allows you to piece together different elements from different popular neuroimaging software packages. In addition to interrogate some information about the images, we will also use the packages [nibabel](https://nipy.org/nibabel/). 

In [1]:
import os
import nibabel as nb
from niwidgets import NiftiWidget
import ants

/Users/davecash/anaconda3/envs/teamcoder_ebm/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps,
/Users/davecash/anaconda3/envs/teamcoder_ebm/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:597: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecat

## Imaging basics
The first step that we are going to do is load in an image and have a quick look at it. There are many different file formats used in medical imaging data, but the two most common are the _Digital Image and COmmunication in Medicine_ ([DICOM](https://www.dicomstandard.org/current))and the _Neuroimaging InFormatics Technology Initiative_ ([NIFTI](https://nifti.nimh.nih.gov/#:~:text=NIfTI%2D1%20is%20a%20new,MRI%20data%20analysis%20software%20packages.). In both case, data is stored as a flat sheet (two-dimensional image data) or a cube (three-dimensional) where individual elements of the data are called pixels (2-D) or voxels (3-D) where one or more numerical values are associated with each element. These values could represent the intensity, a time series of intensities, color values (RGB), vectors, or other different measurements. Associated with this data is _metadata_ which describes key aspects of the data. This metadata could describe, among other things, how the data was acquired, information about the patient, how big the image is, where in physical space of the scanner the image is located, and how data in each pixel is represented.


### Opening an image
Let's first load in an image stored in the Nifti file format into memory and see some basic information about it. One nice thing about Nifti is that you can _compress_ the image using gzip, which makes the image much smaller on your hard drive (roughoy 30-50% for images and up to 90-95% for binary masks), but most software packages can still read it without decomressing it (it will still take up the same amount of memory on your computer when you load it in as part of a program though)


In [ ]:
data_root = "/Users/davecash/Data/IDEAS"
img_name = os.path.join(data_root,"nifti","02-021-11_20151120130726_2.nii.gz")
img = nb.load(img_name)
print(img.shape)


This line of output comes from printing out the field shape from the Nifti image file. It says we have a cube of imaging data in the file that contains 208 rows, 256 columns, and 256 slices.  Now lets find out what this image data contains. I'm going to find a test voxel and ask for its intensity. We typically denote voxel locations as *i*, *j*, and *k*. I have chosen voxel location (100,100,100) which means I want to know the intensity of the voxel at row 100 and column 100 within slice 100. 


In [ ]:
test_voxel = (100,100,100)
img_data = img.get_fdata()
print(img_data[test_voxel])

Python starts counting at **zero**, so this row 100 is actually the 101st row in the matrix. It also means that the indexes used to access the voxels inside the array go frrom rows 0 to 207, not 1 to 208, and columns from 0 to 255, not 1 to 256. Watch what happens if I want to get the first row and column in the last slice if I think Python is counting from 1.

In [ ]:
last_voxel=(1,1,256)
print(img_data[last_voxel])

So what range of intensities are there in this image volume? We can get that by asking what the minimum and maximum intensities are.

In [ ]:
print(img_data.min())
print(img_data.max())

Now we are able to look at voxels, but how do we know where they are in the real world? An MRI of a fruit fly or an elephant could contain 256 slices, but one image would have to represent a much larger size in the real world than the other. What if the elephant was sideways in the scanner, but the fruitfly had its head towards the top of the scanner and its tail at the bottom? Then the rows and columns might mean entirely different anatomical directions!

Embedded in the images metadata are not only the dimensions of each voxel, but also an important transformation, or mapping, that tells us how to convert the voxel location *(i,j,k)* to the real-world coordinates of *(x,y,z)*. Understanding this mapping is very important, as we will be needing this information to align images later. This transformation is stored in the affine field. 

The real-world coordinate system tends to be defined according to the patient. The x-axis tends to go from patient left to patient right, the y axis tends to go from anterior to posterior, and the z-axis goes from top to bottom of the patient. 

In [ ]:
print(img.affine)

What we see here is a four by four matrix. The first 3 rows and columns represent how we *rotate* and *scale* the image voxel to get it into real worlsd space. Here all the numbers off the main diagonal are 0, indicating that there is no rotation going on. That means that the rows of voxels go from left to right in the real world and the columns go from the patients front to the patients back, and finally the slices go from the top of the head down towards the neck. When there is no rotation, than it is really easy to understand how big each indivudal voxel, as the values in the diagonal will indicate how big the voxel is. These represent effectively that each voxel is 1.1 x 1.1 x 1.1mm in size. So when we move one voxel over, we are moving 1.1mm in real world space. 

The first three numbers in the final column that are all below -100 represent what *translation* or shift we need to do to get the voxel into real-world space. The bottom row is always 0,0,0,1 and is used to make the mapping easier. 

How do we get the equivalent real-world coordinates from the voxel location? Through a process of matrix multiplication

In [ ]:
real_world = img.affine.dot(list(test_voxel) + [1])
print(real_world)

So this means that the voxel (100,100,100) represents physical space (-3.8mm. -12.7mm, -24.1mm). When we have different kinds of images acquired in the scanning session or in other scanning sessions, then may have different orientations or different voxel sizes, then it makes it easier to compare and overlay the images together. We will see a bit more abotu that when we discuss registrati0on. Now that we know a bit more about how to traverse the images and what they mean. Let's take a look at one!

In [ ]:
niwidget = NiftiWidget(img_name)
niwidget.nifti_plotter()

## Bias correction
Notice how in the white matter you can see some variation in intensity. It slowly varies from darker areas to lighter areas. This is due to the fact that the static magnetic field, known as B_0, is not perfectly constant everywhere. There are small fluctuations that slowly vary over the volume of the head that cause these subtle differences in intensity.

The good news is that there are many established _bias correction_ techniques that can normalise the intensity across the image. Some of them are integrated in with future processing steps (see tissue segmetnation section below). let's run one of the most common, N3, right now.

In [ ]:
img = ants.image_read(img_name)
img_n3 = ants.n3_bias_field_correction(img)
ants.plot_ortho_stack([img,img_n3])

## Tissue segmentation
Next we are going to run a step of identifying tissue types within the brain. The brain is typically divided up into three primary tissue types: 
1. grey matter (GM),
2. white matter (WM), and
3. cerebrospinal fluid (CSF)
Some of these tissue types can be further broken down (for example cortical GM versus subcortical GM)

## Brain parcellation
Another bit of labelling that is quite helpful is to identify brain regions for each voxel. These are really helpful when you wnat to define a region of interest (ROI) to obtain summary metrics for. This is primarily useful for calculating volumes of each structure, but also for multimodal analyiss when you want a high-resolution anatomical region defined to obtain summary metrics in low-resolution images where it is not possible to identify these regions directly.

## Registration
Registration is used for many pipelines. The main goal is to determine the optimal mapping to align corresponding anatomy between it can be used to align different modalities together (for example PET to MR) to a standard space (such as MNI152) and to